<a href="https://colab.research.google.com/github/zulfanHamka/technicaltestDE_web_berita/blob/main/backtrack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
from datetime import datetime, timedelta
import sys

# Headers untuk request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
}

# Fungsi ambil isi artikel
def get_article_content(link):
    try:
        res = requests.get(link, headers=headers)
        soup = BeautifulSoup(res.text, 'lxml')
        div_content = soup.find('article', class_='detailsContent')
        if not div_content:
            return "Konten tidak ditemukan"
        paragraphs = div_content.find_all('p')
        content = ' '.join([p.get_text(strip=True) for p in paragraphs])
        return content
    except Exception as e:
        print(f"Error saat mengambil artikel: {link} | {e}")
        return ''

# Fungsi validasi input tanggal
def get_validated_date(prompt):
    while True:
        date_str = input(prompt)
        try:
            valid_date = datetime.strptime(date_str, '%Y-%m-%d')
            return valid_date, date_str
        except ValueError:
            print("Format tanggal salah. Silakan gunakan format YYYY-MM-DD.")

# Ambil input tanggal dari pengguna
print("Silakan masukkan rentang tanggal untuk scraping.")
start_date, start_date_str = get_validated_date("Masukkan tanggal mulai (YYYY-MM-DD): ")
end_date, end_date_str = get_validated_date("Masukkan tanggal selesai (YYYY-MM-DD): ")

# Validasi urutan tanggal
if start_date > end_date:
    print("\nError: Tanggal mulai tidak boleh lebih akhir dari tanggal selesai.")
    sys.exit()

current_date = start_date

# List untuk menampung hasil scrape
all_articles_data = []

# Loop utama untuk setiap tanggal
while current_date <= end_date:
    date_to_scrape = current_date.strftime('%Y-%m-%d')
    print(f"\nMemulai scraping untuk tanggal: {date_to_scrape}")

    try:
        # Cari total halaman untuk tanggal ini
        url_first = f'https://www.bisnis.com/index?categoryId=194&date={date_to_scrape}&type=indeks&page=1#'
        res_first = requests.get(url_first, headers=headers)
        soup_first = BeautifulSoup(res_first.text, 'lxml')

        last_page = 1
        paging_element = soup_first.find('p', class_='pagingLabel')
        if paging_element:
            try:
                last_page = int(paging_element.get_text(strip=True).split()[1])
            except (IndexError, ValueError):
                print(f"Tidak dapat memparsing jumlah halaman untuk tanggal {date_to_scrape}.")
        print(f"Total halaman ditemukan: {last_page}")

        # Loop untuk setiap halaman
        for page in range(1, last_page + 1):
            print(f"Scraping halaman {page} dari {last_page}...")
            url = f'https://www.bisnis.com/index?categoryId=194&date={date_to_scrape}&type=indeks&page={page}#'

            res = requests.get(url, headers=headers)
            soup = BeautifulSoup(res.text, 'lxml')
            articles = soup.find_all('div', class_='artItem')

            if not articles:
                print(f"Tidak ada artikel ditemukan di halaman {page}.")
                continue

            # Loop untuk setiap artikel di halaman
            for art in articles:
                try:
                    a_tag = art.find('a')
                    h4_tag = art.find('h4')

                    if not a_tag or not h4_tag:
                        continue

                    title = h4_tag.get_text(strip=True)
                    link = a_tag['href']

                    # Ambil & format tanggal dari URL
                    try:
                        url_parts = link.split('/')
                        date_from_url_str = url_parts[4]
                        year = date_from_url_str[0:4]
                        month = date_from_url_str[4:6]
                        day = date_from_url_str[6:8]
                        time_info = f"{year}-{month}-{day}"
                    except (IndexError, TypeError):
                        time_info = "Tanggal tidak ditemukan di URL"

                    content = get_article_content(link)

                    # Simpan data ke dictionary
                    article_dict = {
                        'Title': title,
                        'Link': link,
                        'Time': time_info,
                        'Content': content
                    }
                    all_articles_data.append(article_dict)

                    print(f"Berhasil: {title}")
                    time.sleep(1) # Jeda antar artikel

                except Exception as e:
                    print(f"Error parsing artikel: {e}")
                    continue

            # Jeda antar halaman
            time.sleep(2)

    except Exception as e:
        print(f"Gagal memproses tanggal {date_to_scrape}: {e}")

    # Lanjut ke tanggal berikutnya
    current_date += timedelta(days=1)


# Simpan hasil ke file JSON
if all_articles_data:
    file_name = f'data_berita_bisnis_{start_date_str}_to_{end_date_str}.json'
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(all_articles_data, f, ensure_ascii=False, indent=4)
    print(f"\nScraping selesai! Data tersimpan di file: {file_name}")
else:
    print("\nScraping selesai, namun tidak ada data yang berhasil dikumpulkan.")

Silakan masukkan rentang tanggal untuk scraping.
Masukkan tanggal mulai (YYYY-MM-DD): 2025-09-20
Masukkan tanggal selesai (YYYY-MM-DD): 2025-09-21

Memulai scraping untuk tanggal: 2025-09-20
Total halaman ditemukan: 1
Scraping halaman 1 dari 1...
Berhasil: Saham Happy Hapsoro dan Garibaldi Thohir Masuk Radar IHSG Pekan Depan
Berhasil: Saham Industri dan Teknologi Jadi Penggerak IHSG Sepekan, Sektor Finansial Tersendat
Berhasil: Investor Asing Tarik Dana dari SBN, SRBI, Saham usai BI Rate Turun
Berhasil: IHSG Berpotensi Tembus 8.246, Saham ASII, PGAS, hingga TLKM Jadi Favorit
Berhasil: Investor Siap-Siap Terima Jatah IPO Merdeka Gold (EMAS), Analis Ramalkan Laba Tahun 2026
Berhasil: Proyeksi Harga Emas Pekan Depan usai Kembali Tembus Rekor Tertinggi, Siap Cuan?
Berhasil: IHSG Berpotensi Menguat Lagi, Simak Rekomendasi Saham BBCA & BBTN Cs Pekan Depan
Berhasil: Ekspektasi dan Peluang Re-Rating Saham GOTO
Berhasil: BRPT, DSSA, hingga TLKM Jadi Pendorong IHSG Tembus Rekor 8.051
Berhasil: S